In [1]:
import requests
from datetime import datetime
import pandas as pd
import plotly.express as px

url = 'https://app.ic-meter.com/icm/oauth/token'
#headers={'Content-Type':'application/x-www-form-urlencoded'}
body={'client_id':'trusted-client',
'grant_type':'password',
'scope':'read',
'username':'parham.barazesh@intellectuallabs.no',
'password':'k2Txff8ZG'}
response = requests.post(
        url,
        body,
        #headers
        )

In [2]:
response.status_code

200

In [3]:
response.json()

{'access_token': '65eca3b8-39b3-4006-a6ed-d95ed31a542a',
 'token_type': 'bearer',
 'refresh_token': 'c23d288e-1b2b-4517-9390-27591598f68d',
 'expires_in': 42297,
 'scope': 'read'}

In [4]:
token=response.json()['access_token']

In [5]:
url='https://app.ic-meter.com/icm/api/buildings/2.0'
body={'extended':'false',
'hide_empty':'false',
'access_token':token}
building_specs = requests.get(
        url,
        body
        )

In [6]:
building_specs.status_code

200

In [7]:
building_specs.json()

[{'buildingId': 912383,
  'name': 'DEG-16(Barcode)',
  'latitude': 59.9082516,
  'longitude': 10.7589495,
  'timezone': 'Europe/Oslo',
  'addressCity': 'Oslo',
  'addressCountry': 'Norge',
  'addressDoor': '16',
  'addressPlace': 'Oslo',
  'addressPostcode': '0191',
  'addressStreet': 'Dronning Eufemias gate',
  'companyIdentification': 'MIRIS  AS',
  'companyName': None,
  'departmentIdentification': 'OSLO',
  'departmentName': None,
  'floorarea': 60000.0,
  'ownership': 'shared'}]

In [8]:
building_id=building_specs.json()[0]['buildingId']

In [9]:
building_id

912383

In [10]:
start_time='2020-01-01T22:00:00Z'
#end_time='2018-12-31T23:00:00Z'
end_time=str(datetime.now())[:19].replace(' ','T')+'Z'
url='https://app.ic-meter.com/icm/api/buildings/2.0/building/indoor/'+str(building_id)+'?start_time='+start_time+'&end_time='+end_time+'&resolution=minute&access_token='+token
response=requests.get(url)
response.status_code

200

In [11]:
url='https://app.ic-meter.com/icm/api/buildings/2.0/building/indoor/'+str(building_id)
body={'start_time':start_time,
'end_time':end_time,
'resolution':'minute',
'access_token':token}
sensor_data = requests.get(
        url,
        body
        )

In [12]:
df=pd.DataFrame(pd.json_normalize(sensor_data.json()).units[0][0])
df.head()

,unitId,name,indoorMeasurements,boxId
0,11419,Utsikten gsm,"{'time': '2020-09-03T13:55:49Z', 'temperature'...",96981941
1,11419,Utsikten gsm,"{'time': '2020-09-03T13:50:49Z', 'temperature'...",96981941
2,11419,Utsikten gsm,"{'time': '2020-09-03T13:45:50Z', 'temperature'...",96981941
3,11419,Utsikten gsm,"{'time': '2020-09-03T13:40:34Z', 'temperature'...",96981941
4,11419,Utsikten gsm,"{'time': '2020-09-03T13:35:50Z', 'temperature'...",96981941


In [13]:
Time=[d['time'] for d in df.indoorMeasurements]

In [14]:
min(Time)

'2020-02-17T15:08:18Z'

In [15]:
df=df[:-1]

In [16]:
df.indoorMeasurements[56957]

{'time': '2020-02-18T18:55:05Z',
 'temperature': 18.42,
 'humidity': 25.22,
 'co2': 449.0,
 'noiseAverage': 32.0,
 'noisePeak': 32.0,
 'light': 0.0}

In [17]:
if sensor_data.status_code==200:
    df=pd.DataFrame(pd.json_normalize(sensor_data.json()).units[0][0])
    df=df[:-1]
    
    Time=[d['time'] for d in df.indoorMeasurements]
    Temperature=[d['temperature'] for d in df.indoorMeasurements]
    Humidity=[d['humidity'] for d in df.indoorMeasurements]
    CO2=[d['co2'] for d in df.indoorMeasurements]
    NoiseAverage=[d['noiseAverage'] for d in df.indoorMeasurements]
    NoisePeak=[d['noisePeak'] for d in df.indoorMeasurements]
    Light=[d['light'] for d in df.indoorMeasurements]

    all_data={'Temperature':pd.DataFrame({'Time':Time,'Temperature':Temperature}),
    'Humidity':pd.DataFrame({'Time':Time,'Humidity':Humidity}),
    'CO2':pd.DataFrame({'Time':Time,'CO2':CO2}),
    'NoiseAverage':pd.DataFrame({'Time':Time,'NoiseAverage':NoiseAverage}),
    'NoisePeak':pd.DataFrame({'Time':Time,'NoisePeak':NoisePeak}),
    'Light':pd.DataFrame({'Time':Time,'Light':Light}),}

    for item in all_data:
        fig = px.line(all_data[item], x=all_data[item].columns[0], y=all_data[item].columns[1], title=item)

        fig.update_xaxes(rangeslider_visible=True,
        rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1 day", step="day", stepmode="backward"),
                    dict(count=7, label="1 week", step="day", stepmode="backward"),
                    dict(count=1, label="1 month", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            ))

        fig.write_html("/home/parham/Documents/codes/factorymind/factorymind/config/static/"+item+".html")
        data=1
else:
    err_msg='Server not responding...'
    data={'error':err_msg}

In [190]:
import json
json.dumps(all_data['Temperature'].to_json())

\\"55841\\":18.69,\\"55842\\":18.68,\\"55843\\":18.64,\\"55844\\":18.64,\\"55845\\":18.64,\\"55846\\":18.66,\\"55847\\":18.68,\\"55848\\":18.69,\\"55849\\":18.71,\\"55850\\":18.73,\\"55851\\":18.73,\\"55852\\":18.75,\\"55853\\":18.75,\\"55854\\":18.77,\\"55855\\":18.77,\\"55856\\":18.77,\\"55857\\":18.8,\\"55858\\":18.75,\\"55859\\":18.78,\\"55860\\":18.79,\\"55861\\":18.78,\\"55862\\":18.77,\\"55863\\":18.78,\\"55864\\":18.8,\\"55865\\":18.8,\\"55866\\":18.8,\\"55867\\":18.8,\\"55868\\":18.78,\\"55869\\":18.77,\\"55870\\":18.75,\\"55871\\":18.73,\\"55872\\":18.73,\\"55873\\":18.7,\\"55874\\":18.64,\\"55875\\":18.59,\\"55876\\":18.54,\\"55877\\":18.48,\\"55878\\":18.41,\\"55879\\":18.3,\\"55880\\":18.22,\\"55881\\":18.13,\\"55882\\":18.07,\\"55883\\":17.97,\\"55884\\":17.91,\\"55885\\":17.86,\\"55886\\":17.82,\\"55887\\":17.77,\\"55888\\":17.71,\\"55889\\":17.67,\\"55890\\":17.62,\\"55891\\":17.57,\\"55892\\":17.53,\\"55893\\":17.5,\\"55894\\":17.48,\\"55895\\":17.47,\\"55896\\":17.46,